In [7]:
import numpy as np

X = np.array([[1,2],[2,3]])
y = np.array([[1],[-1]])

def k(x,xprime,l):
    """Get covariance (using RBF with lengthscale l) between x and xprime"""
    return np.exp(-.5*np.sum((x-xprime)**2)/(l**2))

###### Consider a GP as $\sum{\alpha_i} \text{k}(\mathbf{x_i},\mathbf{x_*})$
where $\mathbf{\alpha} = K^{-1}\mathbf{y}$

We are interested in the greatest gradient.

So we differentiate the whole thing by $x_{*d}$,

Which, for the RBF kernel, gives us:

$$\sum{\frac{\alpha_i}{l^2} (x_{id} - x_{*d}) \text{k}(\mathbf{x_i},\mathbf{x_*})}$$

If the lengthscales are different on different dimensions, then $l$ is the lengthscale of dimension $d$.

To find a bound on the maximum inside a box, with bounds $b_i \leq x_{*i} \leq b_i'$ we do the following steps:

First, we just consider one input $\mathbf{x_i}$ at a time.

1. First consider dimension $d$. The maximum is at location $x_{id} - l \text{sign}({y})$.
2. Test whether this location is within $b_d$ and $b_d'$.
3. If it isn't, compute the value at $b_d$ and $b_d'$ (with all the other values of $x_{*j}$ set to the values of $x_{ij}$).
4. If one or more of the values is positive, select the largest (most positive). The other values of $x_{*j}$ set to the values of $x_{ij}$, or if these lie outside $b_j$ $b_j'$ set to the value of $b_j$ or $b_j'$ closest to $x_{ij}$
5. If both the values are negative, then select the most positive. The other values of $x_{*j}$ should be set to those values of $b_j$ and $b_j'$ that are as far from $x_{ij}$.
 
This gives us the value of that element of the sum which is largest inside the box.

We then repeat 1-5 for each input $x_i$ $(i=1..N)$.

We finally have, for a box, a list of values telling us the largest value the gradient can be for each term in the sum. We simply sum these to get an upper bound on the sum of the elements and hence the gradient in that direction, in that box.

We split the input domain into a series of boxes (note that with the RBF kernel the above gradient will tend to zero at infinity) and find the bound in each. We repeat this for a value of $d=1..D$.

We can finally find the largest change in output a change in one axis can cause. We are interested in a line across the domain in direction of axis $d$ such that the greatest increase occurs, while adding up all the boxes... so imagine we have a 2d space, that is split into 4 boxes,

<pre>
 6acros 3acr
|------|---|
|   2  | 3 |
|------|---|
|   3  | 4 |
|==========>
|------|---|
</pre>

The bounds on the gradient are written in the box. So we can see that the largest change maybe is a line in the bottom two boxes (indicated by the =====>). The first box is width 6 and has max gradient 3 so a change of up to 18 occured there. The second box has a max change of 12. So the total change is 30.

We might want to split the first box in half, and find closer bounds... etc.